In [ ]:
import pymongo
import gridfs

# connect to MongoDB server
client = pymongo.MongoClient("mongodb://localhost:27017/")
# select database
db = client["mydatabase"]
# create GridFS object for the database
fs = gridfs.GridFS(db)

# specify the path to the image file
file_path = "/home/cosmic/WorkSpace/DeepLearnReact/backend/assets/Sekiro.jpg"

# read the contents of the image file
with open(file_path, 'rb') as f:
    contents = f.read()

# store the image file in the database via GridFS
file_id = fs.put(contents, filename="Sekiro.jpg")

# save the file_id in a regular collection
image_collection = db["images"]
image_collection.insert_one({"file_id": file_id})


<img src="/home/cosmic/WorkSpace/DeepLearnReact/backend/assets/help1.png">